> # Cloud Vision API

- 사용하기에 앞서 Cloud Vision API 활성화와 클라이언트 생성을 선행
- 활성화는 console에서 API 사용 설정 가능
- 클라이언트는 `google.cloud.vision`을 통해 생성

In [3]:
!pip install --upgrade google-cloud-vision

Requirement already up-to-date: google-cloud-vision in /usr/local/envs/py3env/lib/python3.5/site-packages (0.34.0)


In [4]:
from google.cloud import vision
from google.cloud.vision import types

- client 인스턴스 만들기

In [5]:
client = vision.ImageAnnotatorClient()


> ## 레이블 검출하기
#### 이미지 지정하고 API 호출하기
- API에 이미지를 전송하려면 로컬 파일을 인코딩해서 요청 매개변수를 포함시키거나, 파일 경로를 지정해야 함
- 라이브러리를 사용하면 별도의 처리를 따로 하지 않아도 됨
- **아래는 로컬파일 또는 GCS 사용 시 API 호출하는 방법임**

In [8]:
# local 파일 사용 시
import io
with io.open('gcpml-book/Part1/seagull.jpg', 'rb') as image_file:
  content = image_file.read()

image = types.Image(content=content)

In [7]:
# GCS 사용시
image = types.Image()
image.source.image_uri = 'gcpml-book/Part1/seagull.jpg'

___
#### 레이블 검출하고 실행하기
- `client`객체에 `label_detection`메서드를 호출하면 API에 요청
- 성공 시 레이블 정보가 리스트로 반환

- 아래와 같이 API에 요청

In [9]:
response = client.label_detection(image = image)

- 반환된 객체에 label_annotations를 보면 다양한 정보가 포함되어 있음

In [11]:
labels = response.label_annotations
labels[0]

mid: "/m/015p6"
description: "bird"
score: 0.9799333214759827
topicality: 0.9799333214759827

- for문을 사용하여 중요 정보를 요약

In [12]:
print ('Labels:')

for i in labels:
  print('Score: {}, Label: {}'.format(round(i.score, 2), i.description))

Labels:
Score: 0.98, Label: bird
Score: 0.94, Label: water
Score: 0.92, Label: seabird
Score: 0.88, Label: beak
Score: 0.87, Label: fauna
Score: 0.84, Label: gull
Score: 0.72, Label: water bird
Score: 0.72, Label: charadriiformes
Score: 0.65, Label: european herring gull
Score: 0.51, Label: feather


> ## 얼굴 검출하기

- 이전과 마찬가지로 클라이언트를 만들고 이미지를 읽어와서 `image` 객체를 생성
- 이번에는 `face_detection`을 통해 API를 요청

In [13]:
with io.open('gcpml-book/Part1/face.jpg', 'rb') as image_file:
  content = image_file.read()
image = types.Image(content=content)

In [14]:
response = client.face_detection(image=image)
face = response.face_annotations

- 반환된 결과는 얼굴 주요 부위의 좌표, 감정, 특징 등의 정보가 포함되어 있음
```
face[0]
>bounding_poly {
>  vertices {
>    x: 512
>    y: 260
>  }
>  vertices {
>    x: 671
>    y: 260
>  }
>  vertices {
>    x: 671
>    y: 445
>  }
>  vertices {
>    x: 512
>    y: 445
>  }
>}
>.
>.
>.
>under_exposed_likelihood: VERY_UNLIKELY
>blurred_likelihood: VERY_UNLIKELY
>headwear_likelihood: VERY_UNLIKELY
```

- 감정은 카테고리 값(int)를 출력하기에 이에 대응하는 튜플 생성 선행

In [15]:
face[0].joy_likelihood

5

In [16]:
likelihood_lists = ['Unknown', 'Very_unlikely', 'Unlikely', 'Possible', 'Likely', 'Very_likely']
print('first face demonstrates "{} joy"'.format(likelihood_lists[face[0].joy_likelihood]))

first face demonstrates "Very_likely joy"


- 검출된 얼굴들의 감정을 모두 출력하자면 다음과 같이 가능

In [17]:
for i in range(len(face)):
  print('face'+str(i+1)+' '+'='*18)
  print('JOY: '+ likelihood_lists[face[i].joy_likelihood])
  print('ANGER: '+ likelihood_lists[face[i].anger_likelihood])
  print('SORROW: '+ likelihood_lists[face[i].sorrow_likelihood])
  print('SURPRISE: '+ likelihood_lists[face[i].surprise_likelihood]+'\n')

face1 ==================
JOY: Very_likely
ANGER: Very_unlikely
SORROW: Very_unlikely
SURPRISE: Very_unlikely

face2 ==================
JOY: Very_likely
ANGER: Very_unlikely
SORROW: Very_unlikely
SURPRISE: Very_unlikely

face3 ==================
JOY: Very_likely
ANGER: Very_unlikely
SORROW: Very_unlikely
SURPRISE: Very_unlikely

face4 ==================
JOY: Very_likely
ANGER: Very_unlikely
SORROW: Very_unlikely
SURPRISE: Very_unlikely

face5 ==================
JOY: Possible
ANGER: Very_unlikely
SORROW: Very_unlikely
SURPRISE: Very_unlikely



- `labels`에 포함된 정보로 얼굴의 요소를 추출할 수도 있으며, 이를 통해 이미지 위에 그림 그리는 것도 가능

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
plt.figure(figsize = (20, 12))
im = Image.open('gcpml-book/Part1/face.jpg', 'r')
plt.imshow(np.asarray(im))
plt.axis('off')